In [13]:
import os
from dotenv import load_dotenv

load_dotenv("../.env")
AUTH_DATA = os.getenv("AUTH_DATA")

In [14]:
import csv
import time
from typing import Optional, Type

from langchain.agents import (
    AgentExecutor,
    create_gigachat_functions_agent,
)
from langchain.agents.gigachat_functions_agent.base import (
    format_to_gigachat_function_messages,
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from langchain_community.chat_models import GigaChat
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [15]:
import os
from getpass import getpass

os.environ["GIGACHAT_CREDENTIALS"] = getpass()
os.environ["GIGACHAT_SCOPE"] = getpass()

### Основной Промт

In [36]:
system = """
Ты ассистент Искуственного Интелекта по научной деятельности, специализирующийся на помощи
исследователям и студентам в поиске и анализе научных статейю. У тебя есть доступ к обширной
базе данных научных публикаций и ты должен помочь пользователям найти статьи, отвечающие их запросам.

Говори только то, что было найдено в базе данных и результатах поиска.
Ты должен помочь пользователю выбрать научные статьи и предоставить необходимую информацию по ним.

Также у тебя есть доступные функции:

- Для поиска научных статей используй syberleninka_search
- Для суммаризации фрагмента научной статьи используй summary_paper
- Для генерации ключевых слов используй generate_paper_keywords
- Для генерации зоголовка статьи по контексту используй generate_paper_title
- Для перефразирования текста статьи используй paraphrase_paper_text


Перед использованием функций, удостоверься, что у пользователя есть все необходимые данные для запроса.
Если данные недоступны или пользователь не предоставил их, спроси у пользователя.

Не пиши одно и тоже пользователю.
Бери данные только из диалога, когда пользователь явно сообщил тебе их. Не придумывай данные сам.
Если каких-то данных не хватает для вызова функции, то нужно спросить данные у пользователя.

Вот описание твоих возможностей: {description}
"""

In [17]:
"""
Дополнительный функционал

Для получения детальной информации по статье используй fetch_paper_details
Для связи с экспертом в области используй contact_expert
Для анализа и обработки данных статьи используй analyze_paper_data



"""

'\nДополнительный функционал\n\nДля получения детальной информации по статье используй fetch_paper_details\nДля связи с экспертом в области используй contact_expert\nДля анализа и обработки данных статьи используй analyze_paper_data\n'

### Tools

In [62]:
class SearchCyberLinikaInput(BaseModel):
    search_query: str = Field(
        description="поисковый запрос пользователя"
    )

class PaperSummaryInput(BaseModel):
    paper_text: str = Field(
        description="поисковый запрос пользователя"
    )

class PaperKeyWordsInput(BaseModel):
    paper_text: str = Field(
        description="исходный текст статьи указанный пользователем без изменений"
    )

class PaperTitleInput(BaseModel):
    paper_text: str = Field(
        description="исходный текст статьи указанный пользователем без изменений"
    )

class ParaphrasePaperInput(BaseModel):
    paper_text: str = Field(
        description="исходный текст статьи указанный пользователем без изменений"
    )


class SearchCyberLinikaTool(BaseTool):
    name = "syberleninka_search"
    description = """
    Выполняет поиск научных статей по входному текстовому запросу пользователя.
    Перед тем как осуществлять поиск, извлеки тему поискового запроса, например:

    Запрос пользователя: "Найди мне статьи сравнивающий LLM модели размером 7B"
    Тема: "LLM модели 7B"

    Запрос пользователя: "Возможности нейронных сетей"
    Тема: "Нейронные сети"
|
    Запрос пользователя: "Перспективы машинного обучения в видеоиграх"
    Тема: "Машинное обучение в видеоиграх"
    """
    args_schema: Type[BaseModel] = SearchCyberLinikaInput

    def _run(
        self,
        search_query: str="",
        run_manager=None,
    ) -> str:
        print(f"Принял поисковый запрос: {search_query}")
        return search_query


class PaperSummaryTool(BaseTool):
    name = "summary_paper"
    description = """
    Выполняет суммаризацию научной статьи.
    Перед тем как осуществлять суммаризацию, извлеки текст который необходимо суммаризировать и верни его, например:

    Запрос пользователя: "Cуммаризируй данный текст: 'Половину шоколада в мире обеспечивает какао, которое растет в западноафриканских странах. Однако там распространяется вирус ...'"
    Параграф для суммаризации: "Половину шоколада в мире обеспечивает какао, которое растет в западноафриканских странах. Однако там распространяется вирус ..."

    Запрос пользователя: "Суммаризируй Деревья какао в Кот-д’Ивуаре и Гане оказались в опасности из-за вируса, который вызывает так называемое опухание побегов и"
    Параграф для суммаризации: Деревья какао в Кот-д’Ивуаре и Гане оказались в опасности из-за вируса, который вызывает так называемое опухание побегов и  ...
    """
    args_schema: Type[BaseModel] = PaperSummaryInput

    def _run(
        self,
        paper_text: str="",
        run_manager=None,
    ) -> str:
        print(f"Принял параграф для суммаризации: {paper_text}")
        return paper_text


class PaperKeyWordsTool(BaseTool):
    name = "generate_paper_keywords"
    description = """
    Выполняет генерацию ключевых слов научной статьи по входному текстовому запросу пользователя.
    Перед тем как осуществлять генерацию ключевых слов, извлеки текст который необходимо суммаризировать и верни его.
    """
    args_schema: Type[BaseModel] = PaperKeyWordsInput

    def _run(
        self,
        paper_text: str="",
        run_manager=None,
    ) -> str:
        print(f"Принял параграф для генерации ключевых слов: {paper_text}")
        return paper_text
    

class PaperTitleTool(BaseTool):
    name = "generate_paper_title"
    description = """
    Выполняет генерацию заголовка научной статьи по входному текстовому запросу пользователя.
    Перед тем как осуществлять генерацию заголовка, извлеки текст для которого необходимо сделать заголовок.
    """
    args_schema: Type[BaseModel] = PaperTitleInput

    def _run(
        self,
        paper_text: str="",
        run_manager=None,
    ) -> str:
        print(f"Принял текст для генерации заголовка статьи: {paper_text}")
        return paper_text
    

class ParaphrasePaperTool(BaseTool):
    name = "paraphrase_paper_text"
    description = """
    Выполняет переформулировку текста научной статьи по входному текстовому запросу пользователя.
    Перед тем как осуществлять переформулировку текста, извлеки текст который необходимо переформулировать.
    """
    args_schema: Type[BaseModel] = ParaphrasePaperInput

    def _run(
        self,
        paper_text: str="",
        run_manager=None,
    ) -> str:
        print(f"Принял текст для генерации заголовка статьи: {paper_text}")
        return paper_text
    

### Tools Testing Promts

In [58]:
"""
# Суммаризация
Суммаризируй По словам авторов исследования, на мучнистых червецов плохо действуют пестициды. Из-за этого фермеры пытаются бороться с вирусом, 
Суммаризируй текст Исследователи с помощью анализа математических данных определили, на каком расстоянии стоит друг от друга высаживать
Сделай суммаризацию "Половину шоколада в мире обеспечивает какао, которое растет в западноафриканских странах. Однако там распространяется вирус, который поражает это растение и его семена."

# Тема
Найди мне статьи сравнивающий LLM модели размером 7B
Гидрофобизация изделий из гипсовых вяжущих одно из направлений расширения сферы их применения в строительстве

# Ключевые слова
Извлеки ключевые слова: Исследователи с помощью анализа математических 
данных определили, на каком расстоянии стоит друг от друга высаживать
привитые и непривитые деревья, чтобы предотвратить распространение вируса
и при этом контролировать затраты на вакцину.

Дай список ключевых слов из текста Половину шоколада в мире обеспечивает какао, которое растет в западноафриканских странах. Однако там распространяется вирус, который поражает это растение и его семена.

# Заголовок 
Создай заголовок для данной статьи Половину шоколада в мире обеспечивает какао, которое растет в западноафриканских странах. Однако там распространяется вирус, который поражает это растение и его семена.
Напиши мне заголовок из статьи тяжелый металл, который способен вызывать неблагоприятные последствия для человеческого организма, включая эндокринные нарушения или остеопороз. В контексте западных диет он считается канцерогеном

# Перефразирование
Перефразируй даннйы текст: Половину шоколада в мире обеспечивает какао, которое растет в западноафриканских странах. Однако там распространяется вирус, который поражает это растение и его семена. 
Перепеши данный текст тяжелый металл, который способен вызывать неблагоприятные последствия для человеческого организма, включая эндокринные нарушения или остеопороз. В контексте западных диет он считается канцерогеном
"""




'\n# Суммаризация\nСуммаризируй По словам авторов исследования, на мучнистых червецов плохо действуют пестициды. Из-за этого фермеры пытаются бороться с вирусом, \nСуммаризируй текст Исследователи с помощью анализа математических данных определили, на каком расстоянии стоит друг от друга высаживать\nСделай суммаризацию "Половину шоколада в мире обеспечивает какао, которое растет в западноафриканских странах. Однако там распространяется вирус, который поражает это растение и его семена."\n\n# Тема\nНайди мне статьи сравнивающий LLM модели размером 7B\nГидрофобизация изделий из гипсовых вяжущих одно из направлений расширения сферы их применения в строительстве\n\n# Ключевые слова\nИзвлеки ключевые слова: Исследователи с помощью анализа математических \nданных определили, на каком расстоянии стоит друг от друга высаживать\nпривитые и непривитые деревья, чтобы предотвратить распространение вируса\nи при этом контролировать затраты на вакцину.\n\nДай список ключевых слов из текста Половину 

In [59]:
llm = GigaChat(
    credentials=AUTH_DATA,
    verify_ssl_certs=False,
    timeout=600,
    model='GigaChat-Pro-preview',
    scope='GIGACHAT_API_CORP'
)

In [60]:
tools = [
    SearchCyberLinikaTool(),
    PaperSummaryTool(),
    PaperKeyWordsTool(),
    PaperTitleTool(),
    ParaphrasePaperTool()
    
]
agent = create_gigachat_functions_agent(llm, tools)

# AgentExecutor создает среду, в которой будет работать агент
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=False, return_intermediate_steps=True
)

In [61]:
chat_history = [SystemMessage(content=system)]
while True:
    user_input = input("Пользователь: ")
    print(f"Пользователь: {user_input}")
    if user_input == "":
        break
    result = agent_executor.invoke(
        {
            "chat_history": chat_history,
            "input": user_input,
        }
    )
    print("\033[93m" + f"Bot: {result['output']}" + "\033[0m")
    chat_history.append(HumanMessage(content=user_input))
    chat_history += format_to_gigachat_function_messages(result["intermediate_steps"])
    chat_history.append(AIMessage(content=result["output"]))
    time.sleep(0.3)

Пользователь: Сделай суммаризацию "Половину шоколада в мире обеспечивает какао, которое растет в западноафриканских странах. Однако там распространяется вирус, который поражает это растение и его семена."
Bot: В статье обсуждается проблема распространения вируса, поражающего растения какао в западноафриканских странах, которые являются основными поставщиками этого продукта в мире.
Пользователь: 
